- 크롤링
    - 내가 작성한 프로그램에 의해서, 웹 드라이버 브라우저 벤더별로 만든 웹드라이버 (핸들링이 가능한 브라우저)를 액세스하여 데이터 수집행위를 하는것
    - 재료
        - 웹 드라이버 (로컬 PC에 존재)
            - 브라우저를 직접 띄워서 처리하는 방식
                - 아나콘다에서 진행
                
            - 브라우저를 백엔드(고스트브라우저)에서 처리하는 방식 (숨김기능 => 리눅스환경)
        - 모듈
            - selenium 
            - 크롬 브라우저도 고스트기능을 사용할 수 있어서 리눅스(서버 스타일)에서 구동이 가능
            - 코랩에서는 불가
     

In [74]:
# 1. 모듈 가져오기
from selenium import webdriver as wd
import platform
import time

In [72]:
platform.system()

'Windows'

In [73]:
# 브라우저 띄우기
if platform.system() == 'Windows':
    driver = wd.Chrome('./chromedriver.exe')
else: # Darwin:
    driver = wd.Chrome('./chromedriver')


In [75]:
# 3. 대상 사이트
target_site = 'http://www.opinet.co.kr/searRgSelect.do'
target_site

'http://www.opinet.co.kr/searRgSelect.do'

In [76]:
# 4. 접속 > get 방식 진행 -> 네트워크 속도, 타겟 사이트의 응답속도등을 고려
# 접속후에는 충분히 대기한다. (명시적, 암묵적, time 모듈을 이용하여 쉬는 방법)
# => 셀레니움을 이용한 크롤링 기술은 네트워크 속도가 아주 빠르거나, 일정한 수준을 유지하는 
# 서버에서 진행, 서버 사양도 괜찮아야한다. (단, 고스트를 사용할때는 관계없음)
# 2회 연속 접속을 통해서 원하는 사이트로 강제 진입
for _ in range(2):
    driver.get(target_site)
    time.sleep(5) # 5초 대기


In [77]:
# 시도 : #SIDO_NM0 -> select
# 1. 시도의 목록을 획득
#     시도 태그 밑에 있는 모든 option 자식들을 찾아라
driver.find_elements_by_css_selector('#SIDO_NM0 > option')

[<selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="1c0cd5a2-643d-4cf1-8a56-e6260bb36fb4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="c34e38c0-2996-44ac-ad1e-ee59f525cda0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="e0aead0f-daf0-4ded-bd99-51e2a8719d02")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="7f084e44-1e33-4ab6-a18a-b64c8256aa9b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="4e00a63c-1a36-4b43-ac57-4c3975cc8193")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="09405f8b-a096-4aed-bf7a-498a32e5b455")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="ce289de5-b809-4656-ab61-65

In [79]:
for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option'):
    # option들 중에서 value가 존재하는 요소만 , 찾는다.
    if option.get_attribute('value'):
        # 실제 다음단계에 필요한 정보 value를 추출
        print(option.get_attribute('value'))
    

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


In [50]:
sidos = [option.get_attribute('value')
        for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
        if option.get_attribute('value')
        ]
sidos


    

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [80]:
sido_sel = driver.find_element_by_id('SIDO_NM0')
print(sido_sel)
for sido in sidos[:2]:
    print(sido)
    # select 태그의 value값을 지정함으로써,시도 선택을 확정할수 있다
    # 이후 이벤트는 사이트에 미리 설정 이벤트 처리에 의해 시군구가 반영된다.
    # 특정 요소에 value를 세팅하는 함수
    sido_sel.send_keys( sido ) # 이벤트 발생
    # 사이트는 내부적으로 ajax를 이용하여 해당 변경된 시도에 일치되는 시군구 정보를 가져온다.
    # 대기
    time.sleep(5)
    

<selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="261a7c67-25fe-46ad-a541-6dbe4e50e01e")>
서울특별시
부산광역시


In [81]:
# 시군구 : #SIGUNGU_NM0 -> select
driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')

[<selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="6cd41bb8-3d73-4f14-8879-85753c8b8155")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="a3df6fb2-ff46-4002-8c4a-20a8fbe7c27b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="baffb411-ea5a-4d23-88f1-5760bc5181de")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="19a7d51c-2c83-4a25-8c4e-5e2694fad79b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="49c332d8-c4af-4fa2-a9bf-c9d34969b5c9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="d1a22150-de63-49fb-9db6-2b2072a44411")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d2ab31d254d8c0cbc21019afc498a5ba", element="060e252a-aba0-4274-ba41-0f

In [95]:
# 시군구 변경 시작
siguns = [option.get_attribute('value')
        for option in driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')
        if option.get_attribute('value')
        ]
siguns

['강서구',
 '금정구',
 '기장군',
 '남구',
 '동구',
 '동래구',
 '부산진구',
 '북구',
 '사상구',
 '사하구',
 '서구',
 '수영구',
 '연제구',
 '영도구',
 '중구',
 '해운대구']

In [100]:
# 시군구 select 태그의 값을 현재 시군구 설정
# 시군구를 선택하면 => 화면이 껌벅 => DOM tree를 새로 구성한다.
# for문에 넣어 매번 구해줘야한다.

for sido in sidos[:2]:
    print(sido)
    sido_sel = driver.find_element_by_id('SIDO_NM0')
    sido_sel.send_keys( sido ) # 이벤트 발생
    time.sleep(3)
    # 시군구 변경 시작
    siguns = [option.get_attribute('value')
        for option in driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')
        if option.get_attribute('value')
        ]
    # 시군구를 돌아가면서 선택
    for sigun in siguns[-2:]:
        print(sigun)
        driver.find_element_by_id('SIGUNGU_NM0').send_keys(sigun)
        time.sleep(3)
        # 엑셀 다운로드 버튼 클릭
        driver.find_element_by_id('glopopd_excel').click()


서울특별시
중구
중랑구
부산광역시
중구
해운대구


In [102]:
# 저장된 파일을 읽어서 다음 단계를 위해서 준비
from glob import glob

In [105]:
# 엑셀 파일을 다운로드 받은 경로
glob('C:/Users/pnu/Downloads/지역_*.xls')

['C:/Users/pnu/Downloads\\지역_위치별(주유소) (1).xls',
 'C:/Users/pnu/Downloads\\지역_위치별(주유소) (2).xls',
 'C:/Users/pnu/Downloads\\지역_위치별(주유소) (3).xls',
 'C:/Users/pnu/Downloads\\지역_위치별(주유소).xls']

In [ ]:
# 조회 : #searRgSelect -> button <= 클릭 이벤트

In [ ]:
# 엑셀 : javascript:fn_excel_download('os_btn'); <= 클릭이벤트
#        glopopd_excel

In [107]:
# 브라우저 닫기 (창닫고, 프로세스 종료)
driver.close()
driver.quit()


WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.88)


In [ ]:
# 파이썬 프로세스 종료
exit()